In [2]:
import numpy as np
import pandas as pd
import keras
from keras.models import Model,Sequential
from keras.layers import Dense, Embedding, Input, Activation
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [3]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
max_features = 25000
maxlen = 100


In [5]:
train['count_unique_word']=train["comment_text"].apply(lambda x: len(set(str(x).split())))
train['count_word']=train["comment_text"].apply(lambda x: len(str(x).split()))

In [6]:
train['word_unique_percent']=train['count_unique_word']*100/train['count_word']

In [7]:
train = train[train['word_unique_percent']>25]
list_sentences_train = train["comment_text"].fillna("na").values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_test = test["comment_text"].fillna("na").values

In [9]:
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_t = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)

In [7]:
inp = Input(shape=(maxlen, ))
embed_size = 128
model = Sequential()
model.add(Embedding(max_features, embed_size,input_length=100))
model.add(Bidirectional(LSTM(50, return_sequences=True)))
model.add(GlobalMaxPool1D())
model.add(Dropout(0.1))
model.add(Dense(50,activation = 'relu' ))
model.add(Dropout(0.1))
model.add(Dense(6, activation ='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])
model.fit(X_t, y, epochs=3, batch_size=100, validation_split=0.15, verbose=0, callbacks=[TQDMNotebookCallback()])


HBox(children=(IntProgress(value=0, description='Training', max=3), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 0', max=135254), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 1', max=135254), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 2', max=135254), HTML(value='')))

In [23]:
y_test = model.predict(X_te)
sample_submission1 = pd.read_csv("sample_submission1.csv")
sample_submission1[list_classes] = y_test
sample_submission1.to_csv("baseline.csv", index=False)

KeyboardInterrupt: 

In [24]:
sample_submission1 = pd.read_csv("sample_submission1.csv")
sample_submission1[list_classes] = y_test
sample_submission1.to_csv("baseline.csv", index=False)